## [week6-lab & Homework #1] airpollution1.txt, airpollution2.txt, mart1.txt, mart2.txt 파일에 대해서 아래 질문에 답하시오. 

1. airpollution1.txt 파일을 읽어서 "wonju" 를 제외한 나머지 도시의 날짜와 PM2포함하는 RDD를 생성하고 내용을 출력하는 프로그램을 작성하라.
2. airpollution1.txt, airpollution2.txt에 대해서 각 도시의 년월 별 PM25의 평균값을 구하고 출력하는 프로그램을 작성하라.
2. airpollution1.txt, airpollution2.txt에 대해서 각 도시별 전체 기간 평균 PM25, PM10 평균값을 구하고 출력하는 프로그램을 작성하라.
3. mart.txt를 입력으로 해서 <실행결과>와 같은 내용의 RDD를 생성하는 프로그램을 작성하라. 

In [1]:
import sys, re
from pyspark.sql import SparkSession
spark_session = SparkSession.builder.master("local").appName("join-dataset-test2").getOrCreate()

In [2]:
ap1 = spark_session.sparkContext.textFile("./data/airpollution1.txt")
ap1.take(5)


['2017.05,5.20,2,chungju',
 '2017.06,5.19,5,chungju',
 '2017.05,5.20,3,jecheon',
 '2017.06,5.19,4,jecheon',
 '2017.05,5.20,3,wonju']

In [3]:
ap1_1=ap1.map(lambda x:x.split(',')).map(lambda x:(x[0],float(x[1]),int(x[2]),x[3]))
ap1_2=ap1_1.filter(lambda x:x[3]!='wonju')
print(ap1_2.take(5))


[('2017.05', 5.2, 2, 'chungju'), ('2017.06', 5.19, 5, 'chungju'), ('2017.05', 5.2, 3, 'jecheon'), ('2017.06', 5.19, 4, 'jecheon')]


In [18]:
ap2 = spark_session.sparkContext.textFile("./data/airpollution2.txt")
ap2_1 = ap2.map(lambda x:x.split(',')).map(lambda x:(x[0],float(x[1]),int(x[2]),x[3]))
print(ap1_1.take(5))
print(ap2_1.take(5))

[('2017.05', 5.2, 2, 'chungju'), ('2017.06', 5.19, 5, 'chungju'), ('2017.05', 5.2, 3, 'jecheon'), ('2017.06', 5.19, 4, 'jecheon'), ('2017.05', 5.2, 3, 'wonju')]
[('2020.05', 6.2, 4, 'chungju'), ('2020.06', 6.21, 5, 'chungju'), ('2020.05', 7.33, 6, 'jecheon'), ('2020.06', 8.67, 5, 'jecheon'), ('2020.05', 3.31, 6, 'wonju')]


In [12]:
ap1_2=ap1_1.keyBy(lambda x:x[0]).mapValues(lambda x:(x[1],1,x[3])).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])).mapValues(lambda x:x[0]/x[1])
print(ap1_2.take(5))

[('2017.05', 5.2), ('2017.06', 5.19)]


In [24]:
ap1_3=ap1_1.keyBy(lambda x:x[3]).mapValues(lambda x:(x[1],1)).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))
print(ap1_3.take(5))
ap2_3=ap2_1.keyBy(lambda x:x[3]).mapValues(lambda x:(x[1],1)).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))
print(ap2_3.take(5))
join = ap1_3.join(ap2_3).mapValues(lambda x:x[0]+x[1]).mapValues(lambda x:(x[0]+x[2],x[1]+x[3])).mapValues(lambda x:x[0]/x[1])
print(join.take(5))

[('chungju', (10.39, 2)), ('jecheon', (10.39, 2)), ('wonju', (10.39, 2))]
[('chungju', (12.41, 2)), ('jecheon', (16.0, 2)), ('wonju', (11.530000000000001, 2))]
[('chungju', 5.7), ('jecheon', 6.5975), ('wonju', 5.48)]


In [307]:
mart1=spark_session.sparkContext.textFile("./data/mart1.csv")
mart2=spark_session.sparkContext.textFile("./data/mart2.csv")
mart1_1=mart1.map(lambda x:x.split(',')).keyBy(lambda x:int(x[0])).mapValues(lambda x: x.drop(x[0]))#.flatMap(lambda x:x.split(','))
mart2_1=mart2.map(lambda x:x.split(','))
#cogroup1=mart1_1.cogroup(mart2_1).mapValues(lambda x:[item for sublist in x for item in sublist])
print(mart1_1.take(10))


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 462.0 failed 1 times, most recent failure: Lost task 0.0 in stage 462.0 (TID 407) (15e8087bb5c5 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/rdd.py", line 2849, in takeUpToNumLeft
    yield next(iterator)
          ^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py"